In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from Image_Functions import slicing,  crop_to_size
from datasetModule import Set
from torch import nn
from Dice_Loss import DiceLoss
import Model_Aske

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


Most of the functionallity is stored in module files. 
The data consist of Images with 3 channels and segmentation images with 2 channels.


In [2]:
#hyper parameters
batch_size = 2
learning_rate = 0.01
num_epochs = 4

In [3]:
"Need to specify the local path on computer"
dir_path = "../Cropped_Task3/"

'Splitting the data into 30% test and 70% training.'
train_set, test_set = train_test_split(Set(dir_path, sub_dir = 'crop_sub-23'), test_size=0.3, random_state=25)

#X_train, X_test = crop_images_to_brain(X_train), crop_images_to_brain(X_test)
size = (256,288,176)
train_set = crop_to_size(train_set, size)
test_set = crop_to_size(test_set, size)

'Load training and test set, batch size my vary'
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)
test_set = None
train_set = None

../Cropped_Task3/crop_sub-233
(275, 350, 168)
segmentation shape:  torch.Size([1, 275, 350, 168])
../Cropped_Task3/crop_sub-232
(281, 330, 172)
segmentation shape:  torch.Size([1, 281, 330, 172])
../Cropped_Task3/crop_sub-234
(281, 349, 172)
segmentation shape:  torch.Size([1, 281, 349, 172])
../Cropped_Task3/crop_sub-230
(281, 358, 182)
segmentation shape:  torch.Size([1, 281, 358, 182])
../Cropped_Task3/crop_sub-231
(322, 340, 185)
segmentation shape:  torch.Size([1, 322, 340, 185])


In [4]:
'Run the CNN'
model = Model_Aske.CNN(3,base_features=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)




In [8]:
examples = iter(train_loader)
samples = examples.next()

try:
    while (samples['seg'][0][0].max() == 0) or (samples['seg'][0][0].max() == 0):
        samples = examples.next()
except StopIteration:
    print('Iteration is shit')
data = samples['data']
seg = samples['seg']

print(seg.max())
print(seg.shape)
print(data.shape)
out = model(data)



tensor(1., dtype=torch.float64)
torch.Size([1, 1, 256, 288, 176])
torch.Size([1, 3, 256, 288, 176])
Final =  torch.Size([1, 1, 256, 288, 176])


In [10]:

loss_func_1 = nn.BCELoss()
loss_func_2 = DiceLoss()
#target = torch.squeeze(seg, 0).type(torch.LongTensor)
#target = seg.resize_(1,256,288,176).type(torch.LongTensor)
seg = seg.float()
sigmoid = nn.Sigmoid()
out_sig = sigmoid(out)
print(out_sig.max())
loss_1 = loss_func_1(out_sig, seg)
print(loss_1)
loss_2 = loss_func_2(out, seg)
print(loss_2) 

torch.Size([1, 1, 256, 288, 176])
tensor(27864.0117, grad_fn=<MaxBackward1>)
tensor(1., grad_fn=<MaxBackward1>)
tensor(1.7631, grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([1, 1, 256, 288, 176])
tensor(1., grad_fn=<MaxBackward1>)
tensor(1.0000, grad_fn=<RsubBackward1>)


Notes on how things are going: 


BCELoss requires no squeezing, where as Cross Entropy requires the feature dimension to be squeezed

In [ ]:

for epoch in range(num_epochs):
    for i, image_set in enumerate(train_loader):
        image = image_set['data'].to(device)
        labels = image_set['seg'].to(device)
        outputs = model(image)

        print("outputs shape = ", outputs.shape)
        print("squeeze = ", torch.squeeze(labels,1).shape)
        loss = criterion(outputs, labels)
        print(1)
        optimizer.zero_grad()
        print(2)
        loss.backward()
        print(3)
        optimizer.step()
        print(4)
        if (i+1) % 1 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

In [ ]:
print(out_img[0][0].shape)
img = out_img[0][0]
img = img.detach().numpy()
for imgur in image[0]:
    slicing(imgur)
#slicing(img)